In [1]:
import pandas as pd
import numpy as np
from itertools import product
from ast import literal_eval
import seaborn as sns
import random
import matplotlib.pyplot as plt
import copy

In [3]:
pubs_author_funder = pd.read_csv("..//..//..//data//Data//DerivedData//CleanedRawData//pubs_author_funder.csv")
#pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(literal_eval)
pubs_author_funder['funding_units'] = pubs_author_funder['funder_country'].apply(lambda x: len(x))

In [4]:
pubs_funder_fair = copy.deepcopy(pubs_author_funder[['id','funder_country','funding_units']])
pubs_funder_fair['author_country']=pubs_funder_fair['funder_country']
pubs_funder_fair.head()

,id,funder_country,funding_units,author_country
0,30507292,[Turkey],1,[Turkey]
1,30574690,"[United States, United States, United States, ...",4,"[United States, United States, United States, ..."
2,30574691,[Russia],1,[Russia]
3,30585095,"[Spain, Spain, Spain, Spain, Spain, Spain]",6,"[Spain, Spain, Spain, Spain, Spain, Spain]"
4,30585097,[Spain],1,[Spain]


In [5]:
def generate_null(data):
    cntry_cntry_explodelist = []
    for index, row in data.iterrows():
        units = row['funding_units']
        author_country = row['author_country']
        funding_country=row['funder_country']
        num_author = len(author_country)
        num_funder=len(funding_country)
        for author_c in author_country:
            for funder_c in funding_country:
                credit = units/(num_author*num_funder)
                templist = [funder_c,author_c, credit]
                cntry_cntry_explodelist.append(templist)
                
    cntry_cntry_null = pd.DataFrame(
        cntry_cntry_explodelist, columns=['source','target','value'])
    cntry_cntry_null = cntry_cntry_null.groupby(['source','target'])['value'].sum().reset_index()
    cntry_cntry_null = cntry_cntry_null.pivot_table(
        index='source',columns='target', values='value', fill_value=0)
    return cntry_cntry_null


In [6]:
funder_fair_matrix=generate_null(pubs_funder_fair)